In [ ]:
import data_utils as dutils
import model_utils as mutils
import numpy as np
import torch
import distributions as distr
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
# continuous optim with completely continuous tissue

# data
datamodule = dutils.Prostate_cancer_2d()
X,y = datamodule.get_data()
tissue = dutils.Tissue_continuous(X, y, slice_radius = 1)
torch.autograd.set_detect_anomaly(True)

# distributions
r_prior = torch.tensor([3.0])
mu = torch.tensor([torch.log(r_prior),0.,0.,0.])
std = torch.tensor([1.,10.,10.,1.])
prior = distr.Circle_prior(mu=mu,std=std)
predictive = distr.Circle_predictive()

# optimization
eig_calc_method = mutils.eig_PCE
eig_optim_method = mutils.eig_cont_optim
n_designs = 4
optim_args = {
    'alpha': ('minmax', -torch.pi, torch.pi),
    'y_intercept': ('minmax', -1, 1),
}

experiment, thetas_MAP, metric_values, metric_names = mutils.OED_fit(
    tissue, prior,predictive,n_designs,eig_method,eig_optim_method,
    print_every=50, verbose='plot',

    # eig optim
    n_steps_optim = 150,

    # eig method
    n_outer = 50,
    n_inner = 20,

    # variaional inference
    n_steps_varinf=1200,
)